In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
data = pd.read_csv("Final Res.csv")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data


,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,상일동,상일동역 도전최강달인왕만두
1,이때,한식,한식/백반/한정식,서울특별시 마포구 동교로38안길 7,경의중앙 홍대입구,360.468770,홍대입구,홍대입구역 이때
2,L.A.D,한식,한식/백반/한정식,서울특별시 마포구 잔다리로3안길 23,2호선 홍대입구,583.594176,홍대입구,홍대입구역 L.A.D
3,제이씨에스푸드,한식,한식/백반/한정식,서울특별시 관악구 신림로14길 3,2호선 봉천,860.540255,봉천,봉천역 제이씨에스푸드
4,비지트,한식,한식/백반/한정식,서울특별시 서초구 동광로18길 82,2호선 방배,810.975643,방배,방배역 비지트
...,...,...,...,...,...,...,...,...
135805,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,방화,방화역 얼음과자
135806,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,상일동,상일동역 오이소서울
135807,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,굽은다리,굽은다리역 얼음왕국명일14
135808,얼음과자,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로83길 6,5호선 상일동,1380.011573,상일동,상일동역 얼음과자


In [3]:
items = data["검색어"].head(7)
items

0    상일동역 도전최강달인왕만두
1          홍대입구역 이때
2       홍대입구역 L.A.D
3       봉천역 제이씨에스푸드
4           방배역 비지트
5       총신대입구역 쓰리에프
6            종각역 채움
Name: 검색어, dtype: object

In [4]:

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}


In [ ]:
count=0
for item in items:
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)

In [5]:
driver = webdriver.Chrome(r'C:/Users/user/Desktop/dd/Data_Handling/chromedriver.exe')




count = 0
for item in items:
    # 리뷰가 없을 때의 코드
    
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        content_url = reviewbutton.get_attribute("href")    
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img ') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                print(user_review)
                print(rating)
                print(img_url)
                print(user_id)
                print(timestamp)
                row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating,"Timestamp":timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row,ignore_index=True)
                time.sleep(1)
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(2)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img ') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating,"Timestamp":timestamp}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row,ignore_index=True)
                        time.sleep(1)
        except:
            print("더 이상 리뷰 존재 X")
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")
        




식당 존재 x
식당 존재


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: use driver.switch_to.window instead


5
[<span>감사했습니다,</span>]
[<em class="num_rate">5<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="http://k.kakaocdn.net/dn/bgkXfY/btqy33qI8Xx/RtMVlh59sev2mcXP56w7TK/profile_640x640s.jpg" data-platform="kakaomap" data-userid="13gfkas" data-username="서지원" href="javascript:void(0);">서지원</a>]
[<span class="time_write">2021.01.21.</span>]
[<span>사장님 바뀜 ㅠㅠㅠㅠ</span>]
[<em class="num_rate">5<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="https://p.kakaocdn.net/th/talkp/wlZJQtZsda/KZ6vJIQtXkBFy4ntZluIH1/pv1d4x_640x640_s.jpg" data-platform="kakaomap" data-userid="ei79fa" data-username="Tuned by' SC" href="javascript:void(0);">Tuned by' SC</a>]
[<span class="time_write">2020.11.29.</span>]
[<span>일단 사람이 별로 없고!! "이때의 라떼"맛있음. 근데 양이 적어서 너무 아쉽 ㅜㅜㅜㅜㅜ맛잇어서 거의 두모금만에 다 마신듯.. 크로아상 짱 작은데 같이 주는 딸기크림이랑 먹으니까 맛있엇다!! 초코볼도 줌</span>]
[<em class="num_rate">4<span class="screen_out">점</span></em>]
[<img alt="포토뷰어" class="img_prof

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: use driver.switch_to.window instead


식당 존재 x
식당 존재 x
식당 존재
1
[<span>카프레제 샌드위치 시켜봤는데 충격적으로 맛있음...와....감동쓰ㅠㅠ</span>]
[<em class="num_rate">5<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="https://p.kakaocdn.net/th/talkp/wlW0QXOMcx/yJjMVSPt3ZLsNX9CKNaiK0/fumkyk_640x640_s.jpg" data-platform="kakaomap" data-userid="1fr1u37" data-username="Jungeun" href="javascript:void(0);">Jungeun</a>]
[<span class="time_write">2021.01.23.</span>]
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자


KeyboardInterrupt: 

In [6]:
user_id[0].get('data-userid')

'1fr1u37'

In [7]:
user_id

[<a class="link_user" data-ismy="false" data-ori="https://p.kakaocdn.net/th/talkp/wlW0QXOMcx/yJjMVSPt3ZLsNX9CKNaiK0/fumkyk_640x640_s.jpg" data-platform="kakaomap" data-userid="1fr1u37" data-username="Jungeun" href="javascript:void(0);">Jungeun</a>]

In [8]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,13gfkas,홍대입구역 이때,"[5, [점]]",[2021.01.21.]
1,ei79fa,홍대입구역 이때,"[5, [점]]",[2020.11.29.]
2,tqv4dr,홍대입구역 이때,"[4, [점]]",[2020.09.14.]
3,10m615r,홍대입구역 이때,"[3, [점]]",[2020.08.21.]
4,v6a12s,홍대입구역 이때,"[4, [점]]",[2020.08.07.]
5,13gfkas,홍대입구역 이때,"[5, [점]]",[2021.01.21.]
6,ei79fa,홍대입구역 이때,"[5, [점]]",[2020.11.29.]
7,tqv4dr,홍대입구역 이때,"[4, [점]]",[2020.09.14.]
8,10m615r,홍대입구역 이때,"[3, [점]]",[2020.08.21.]
9,v6a12s,홍대입구역 이때,"[4, [점]]",[2020.08.07.]
